In [1]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
import numpy as np

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
#Population pull
population_url = race_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,P001001&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
population = requests.get(population_url).json()

In [3]:
population_df = pd.DataFrame(population)
population_df.columns = population_df.iloc[0]
population_df = population_df.reindex(population_df.index.drop(0))
population_df = population_df.rename(columns={"NAME":"State","P001001":"population","state":"count"})
population_df = population_df[['State','population']]
population_df.head(52)
population_df.to_excel("pop_data.xlsx")

In [4]:
#astype(float).sum(axis=1)
population_df["population"]= population_df["population"].astype(int) 

In [5]:
#Append row with US which is sum of entire populations
total = population_df.apply(np.sum)
total['State'] = 'US'
population_df = population_df.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)

In [6]:
population_df.tail(2)

,State,population
51,Puerto Rico,3725789
52,US,312471327


In [7]:
#Race pull
race_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,PCT012C001,PCT012D001,PCT012B001,PCT012H001,PCT012E001,PCT012F001,PCT012G001,PCT012A001&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
race = requests.get(race_url).json()

In [8]:
race_df = pd.DataFrame(race)
race_df.columns = race_df.iloc[0]
race_df = race_df.reindex(race_df.index.drop(0))
race_df = race_df.rename(columns={"NAME":"State","PCT012C001":"american_indian_and_alaska_native","PCT012D001":"asian","PCT012B001":"black_or_african_american","PCT012H001":"hispanic_or_latino","PCT012E001":"native_hawaiian_other_pacific_islander","PCT012F001":"some_other_race","PCT012G001":"two_or_more_races","PCT012A001":"white"})
del race_df['state']
race_df.tail(2)

,State,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_other_pacific_islander,some_other_race,two_or_more_races,white
51,Wyoming,13336,4426,4748,50231,427,17049,12361,511279
52,Puerto Rico,19839,6831,461498,3688455,370,289905,122246,2825100


In [9]:
#Change columns to int
race_df["american_indian_and_alaska_native"]= race_df["american_indian_and_alaska_native"].astype(int)
race_df["asian"]= race_df["asian"].astype(int)
race_df["black_or_african_american"]= race_df["black_or_african_american"].astype(int)
race_df["hispanic_or_latino"]= race_df["hispanic_or_latino"].astype(int)
race_df["native_hawaiian_other_pacific_islander"]= race_df["native_hawaiian_other_pacific_islander"].astype(int)
race_df["some_other_race"]= race_df["some_other_race"].astype(int) 
race_df["two_or_more_races"]= race_df["two_or_more_races"].astype(int) 
race_df["white"]= race_df["white"].astype(int) 

In [10]:
#Append row with US which is sum of entire populations
total_race = race_df.apply(np.sum)
total_race['State'] = 'US'
race_df = race_df.append(pd.DataFrame(total_race.values, index=total_race.keys()).T, ignore_index=True)

In [11]:
race_df.tail(2)

,State,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_other_pacific_islander,some_other_race,two_or_more_races,white
51,Puerto Rico,19839,6831,461498,3688455,370,289905,122246,2825100
52,US,2952087,14681083,39390817,54166049,540383,19397273,9131319,226378365


In [12]:
#Household Size pull
house_size_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H013002,H013003,H013004,H013005,H013006,H013007,H013008&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
size = requests.get(house_size_url).json()

In [13]:
household_size_df = pd.DataFrame(size)
household_size_df.columns = household_size_df.iloc[0]
household_size_df = household_size_df.reindex(household_size_df.index.drop(0))
household_size_df = household_size_df.rename(columns={"NAME":"State","H013002":"1_persons_or_more","H013003":"2_persons_or_more","H013004":"3_persons_or_more","H013005":"4_persons_or_more","H013006":"5_persons_or_more","H013007":"6_persons_or_more","H013008":"7_persons_or_more"})
del household_size_df['state']
household_size_df.tail(2)

,State,1_persons_or_more,2_persons_or_more,3_persons_or_more,4_persons_or_more,5_persons_or_more,6_persons_or_more,7_persons_or_more
51,Wyoming,63480,83568,33043,26654,12444,4825,2865
52,Puerto Rico,327560,391452,280818,227988,99560,31434,17719


In [14]:
#Function to sum columns 
def sum_household_by_column(frame, new_col_name, list_of_cols_to_sum):
    frame[new_col_name] = frame[list_of_cols_to_sum].astype(int).sum(axis=1)
    return(frame)

#Create new columns with total of combined
household_size_df = sum_household_by_column(household_size_df, '1_or_2_persons', ['1_persons_or_more','2_persons_or_more'])
household_size_df = sum_household_by_column(household_size_df, '3_or_4_persons', ['3_persons_or_more','4_persons_or_more'])
household_size_df = sum_household_by_column(household_size_df, '5_or_more_persons', ['5_persons_or_more','6_persons_or_more','7_persons_or_more'])
household_size_df.tail(2)

,State,1_persons_or_more,2_persons_or_more,3_persons_or_more,4_persons_or_more,5_persons_or_more,6_persons_or_more,7_persons_or_more,1_or_2_persons,3_or_4_persons,5_or_more_persons
51,Wyoming,63480,83568,33043,26654,12444,4825,2865,147048,59697,20134
52,Puerto Rico,327560,391452,280818,227988,99560,31434,17719,719012,508806,148713


In [15]:
#DF with only new totaled sums
household_size_df = household_size_df[['State','1_or_2_persons','3_or_4_persons','5_or_more_persons']]
household_size_df.head(2)

,State,1_or_2_persons,3_or_4_persons,5_or_more_persons
1,Alabama,1148987,568267,166537
2,Alaska,149925,76015,32118


In [16]:
#household_size_df['State'].to_json(orient = "records")

In [17]:
#household_size_df['1_or_2_persons'].to_json(orient = "records")

In [18]:
#household_size_df['3_or_4_persons'].to_json(orient = "records")

In [19]:
#household_size_df['5_or_more_persons'].to_json(orient = "records")

In [21]:
#Append row with US which is sum of entire household sizes
total_size = household_size_df.apply(np.sum)
total_size['State'] = 'US'
household_size_df = household_size_df.append(pd.DataFrame(total_size.values, index=total_size.keys()).T, ignore_index=True)

In [22]:
household_size_df.tail(3)

,State,1_or_2_persons,3_or_4_persons,5_or_more_persons
50,Wyoming,147048,59697,20134
51,Puerto Rico,719012,508806,148713
52,US,70166549,34892037,13034237


#API to pull income
https://api.census.gov/data/2010/acs/acs5/variables.html

#Variable used
B06011_002E	Estimate!!Median income in the past 12 months!!Total!!Born in state of residence	MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2010 INFLATION-ADJUSTED DOLLARS) BY PLACE OF BIRTH IN THE UNITED STATES

api.census.gov/data/2010/acs/acs5?key=YOUR KEY GOES HERE&get=B02001_001E,NAME&for=state:06,36

In [23]:
#Median Income last 12 months pull
income_url = "https://api.census.gov/data/2010/acs/acs5?key=59a41ecf0dff091bd991b35c87176ffd56b8107d&get=B06011_002E,NAME&for=state:*"
income = requests.get(income_url).json()

In [24]:
income_df = pd.DataFrame(income)
income_df.columns = income_df.iloc[0]
income_df = income_df.reindex(income_df.index.drop(0))
income_df = income_df.rename(columns={"NAME":"State","B06011_002E":"med_income_ltm"})
income_df = income_df[['State','med_income_ltm']]
income_df.tail(2)

,State,med_income_ltm
51,Wyoming,25498
52,Puerto Rico,None


In [25]:
income_df['med_income_ltm'].fillna(0, inplace=True)
income_df.tail()

,State,med_income_ltm
48,Washington,27280
49,West Virginia,20071
50,Wisconsin,26954
51,Wyoming,25498
52,Puerto Rico,0


In [26]:
#Change columns to int
income_df["med_income_ltm"]= income_df["med_income_ltm"].astype(int)
#income_df.dtypes

In [27]:
#Append row with US which is sum of entire household sizes
total_income = income_df.apply(np.sum)
total_income['State'] = 'US'
income_df = income_df.append(pd.DataFrame(total_income.values, index=total_income.keys()).T, ignore_index=True)

In [28]:
income_df.tail()

,State,med_income_ltm
48,West Virginia,20071
49,Wisconsin,26954
50,Wyoming,25498
51,Puerto Rico,0
52,US,1245145


In [30]:
#income_df['State'].to_json(orient = "records")

In [32]:
#income_df['med_income_ltm'].to_json(orient = "records")

In [33]:
#Age pull
age_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,P013003,P013002&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
age = requests.get(age_url).json()

In [35]:
age_df = pd.DataFrame(age)
age_df.columns = age_df.iloc[0]
age_df = age_df.reindex(age_df.index.drop(0))
age_df = age_df.rename(columns={"NAME":"State","P013003":"med_age_female","P013002":"med_age_male"})
del age_df['state']
age_df.tail(2)

,State,med_age_female,med_age_male
51,Wyoming,37.7,36.0
52,Puerto Rico,38.6,35.1


In [36]:
age_df['med_age_female'].astype(float)
age_df['med_age_male'].astype(float)
#age_df.med_age_female = age_df.med_age_female.round()
age_df.dtypes

0
State             object
med_age_female    object
med_age_male      object
dtype: object

In [37]:
#Change columns to int
#age_df['med_age_female'].astype(str).astype(float)
#age_df['med_age_male'].astype(str).astype(float)
#age_df["med_age_female"]= age_df["med_age_female"].astype(float)
#age_df["med_age_male"]= age_df["med_age_male"].astype(float)

In [38]:
age_df.tail()

,State,med_age_female,med_age_male
48,Washington,38.3,36.2
49,West Virginia,42.6,40.1
50,Wisconsin,39.6,37.3
51,Wyoming,37.7,36.0
52,Puerto Rico,38.6,35.1


In [39]:
#age_df.to_json(orient = "records")

In [40]:
#Occupancy pull
occupancy_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H004003,H004002,H004004&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
occupancy = requests.get(occupancy_url).json()

In [41]:
occupancy_df = pd.DataFrame(occupancy)
occupancy_df.columns = occupancy_df.iloc[0]
occupancy_df = occupancy_df.reindex(occupancy_df.index.drop(0))
occupancy_df = occupancy_df.rename(columns={"NAME":"State","H004003":"owned_free_clear","H004002":"owned_mortgage","H004004":"rent"})
del occupancy_df['state']
occupancy_df.tail()

,State,owned_free_clear,owned_mortgage,rent
48,Washington,428656,1245264,946156
49,West Virginia,267491,293522,202818
50,Wisconsin,469459,1082099,728210
51,Wyoming,57296,99781,69802
52,Puerto Rico,554508,431657,390366


In [42]:
#Change columns to int
occupancy_df["owned_free_clear"]= occupancy_df["owned_free_clear"].astype(int)
occupancy_df["owned_mortgage"]= occupancy_df["owned_mortgage"].astype(int)
occupancy_df["rent"]= occupancy_df["rent"].astype(int)

In [43]:
#Append row with US which is sum of entire household sizes
total_occupancy = occupancy_df.apply(np.sum)
total_occupancy['State'] = 'US'
occupancy_df = occupancy_df.append(pd.DataFrame(total_occupancy.values, index=total_occupancy.keys()).T, ignore_index=True)

In [44]:
occupancy_df.tail()

,State,owned_free_clear,owned_mortgage,rent
48,West Virginia,267491,293522,202818
49,Wisconsin,469459,1082099,728210
50,Wyoming,57296,99781,69802
51,Puerto Rico,554508,431657,390366
52,US,23561152,53411087,41120584


In [45]:
#Merge data-frames
merged_df = population_df.merge(race_df.merge(household_size_df, on='State'), on='State') 
merged_df_2 = income_df.merge(occupancy_df, on='State') 
merged_final = merged_df.merge(merged_df_2, on="State")

In [47]:
merged_final.tail(3)

,State,population,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_other_pacific_islander,some_other_race,two_or_more_races,white,1_or_2_persons,3_or_4_persons,5_or_more_persons,med_income_ltm,owned_free_clear,owned_mortgage,rent
50,Wyoming,563626,13336,4426,4748,50231,427,17049,12361,511279,147048,59697,20134,25498,57296,99781,69802
51,Puerto Rico,3725789,19839,6831,461498,3688455,370,289905,122246,2825100,719012,508806,148713,0,554508,431657,390366
52,US,312471327,2952087,14681083,39390817,54166049,540383,19397273,9131319,226378365,70166549,34892037,13034237,1245145,23561152,53411087,41120584


In [48]:
merged_final.to_excel("census_data.xlsx")